In [2]:
import os
import time
import copy
import numpy as np
import pandas as pd
import networkx as nx
from datetime import datetime
import argparse
from argparse import ArgumentParser
import itertools
from multiprocessing import Process, Queue
import multiprocessing as mp

def compareAD(M1,M2): #M1 is the ground truth
    error_pairs=[]
    n_adpairs=0
    for i in range(M1.shape[1]):
#        print(i)
        for j in range(i,M1.shape[1]):
            cap1=M1[:,i]*M1[:,j]
            cap2=M2[:,i]*M2[:,j]
            if (np.sum(cap1)>0 and np.sum(M1[:,i]) != np.sum(M1[:,j])):
                n_adpairs=n_adpairs+1
                if (np.sum(cap2)==0):
                    error_pairs.append([i,j])
                else:
                    if (np.sum(M1[:,j])>np.sum(M1[:,i]) and np.sum(M2[:,j])<=np.sum(M2[:,i])):
                        error_pairs.append([i,j])
                    else:
                        if (np.sum(M1[:,i])>np.sum(M1[:,j]) and np.sum(M2[:,i])<=np.sum(M2[:,j])):
                            error_pairs.append([i,j])
                        #print(i,j,sum(M1[:,i]),sum(M1[:,j]),sum(M2[:,i]),sum(M2[:,j]))
    print('Number of AD pairs = ',n_adpairs,"errors : ",len(error_pairs), "AD score = ", 1 - len(error_pairs)/n_adpairs)
    return (1 - len(error_pairs)/n_adpairs)                

def compareDF(M_orj,M_rec):
    error_pairs=[]
    d_pairs=0
    for i in range(M_orj.shape[1]):
        for j in range(i,M_orj.shape[1]):
            cap1=M_orj[:,i]*M_orj[:,j]
            cap2=M_rec[:,i]*M_rec[:,j]
            if np.sum(cap1)==0:
                d_pairs=d_pairs + 1
                if np.sum(cap2)>0:
                    error_pairs.append([i,j])
    print("Number of Diff pairs = ",d_pairs, "errors :",len(error_pairs), "score :", 1-len(error_pairs)/d_pairs)
    return (1-len(error_pairs)/d_pairs)

def WriteTfile(filename,matrix,filename2): # writes matrix output as an integer matrix
    df_input = pd.read_csv(filename2, sep='\t', index_col=0)
    matrix_output=matrix.astype(int)
    df_output = pd.DataFrame(matrix_output)
    df_output.columns = df_input.columns
    df_output.index = df_input.index
    df_output.index.name = "cellIDxmutID"
    df_output.to_csv(filename, sep="\t")
def ReadFasis(filename): # reads a matrix from file and returns it in BOOL type
    df = pd.read_csv(filename, sep='\t', index_col=0)
    M=df.values
    return M        

def AddNAFNFP(NA,FN,FP,groundfile,outputfile):
    M_ground=ReadFasis(groundfile).astype(int)
    M_out=M_ground.copy()
    size_M=M_ground.shape[0]*M_ground.shape[1]
    k_na=np.floor(NA*size_M).astype(int)
    k_fn=np.floor(FN*np.sum(M_ground==1)).astype(int)
    k_fp=np.floor(FP*np.sum(M_ground==0)).astype(int)
    print(np.sum(M_ground==1),np.sum(M_ground==0))
    print("Required NA/FN/FP",k_na,k_fn,k_fp," Number of ones in the matrix : ",np.sum(M_ground))
    
    while k_na+k_fp+k_fn >0:
        
        sel=np.random.choice(size_M,k_na+k_fp+k_fn, replace=False)
        for i in sel:
            iy=i//M_ground.shape[0]
            ix=i%M_ground.shape[0]
            if M_ground[ix,iy]==M_out[ix,iy]:
                if k_na>0:
                    M_out[ix,iy]=3
                    k_na=k_na-1
                else:
                    
                    if k_fp>0 and M_out[ix,iy]==0:
                        M_out[ix,iy]=1
                        k_fp=k_fp-1
                    else:
                        if k_fn>0 and M_out[ix,iy]==1:
                            M_out[ix,iy]=0
                            k_fn=k_fn-1
    print("Sanity Check ", np.sum(M_out==3),np.sum(M_ground>M_out), np.sum(M_out>M_ground)-np.sum(M_out==3))    
    WriteTfile(outputfile,M_out,groundfile)

def Add_Pwise_NAFNFP(p_na,p_fn,p_fp,groundfile,outputfile):
    M_ground=ReadFasis(groundfile).astype(int)
    M_out=M_ground.copy()
    size_M=M_ground.shape[0]*M_ground.shape[1]
    k_na=np.floor(p_na*size_M).astype(int)
    k_fn=np.floor(p_fn*np.sum(M_ground==1)).astype(int)
    k_fp=np.floor(p_fp*np.sum(M_ground==0)).astype(int)
    print(np.sum(M_ground==1),np.sum(M_ground==0))
    print("Required NA/FN/FP ",k_na,k_fn,k_fp)
    
    for ix in range(M_out.shape[0]):
        for iy in range(M_out.shape[1]):
            if np.random.rand() < p_na:
                M_out[ix,iy]=3
            else:
                if M_out[ix,iy]==1 and np.random.rand() < p_fn:
                    M_out[ix,iy]=0
                else: 
                    if M_out[ix,iy]==0 and np.random.rand() < p_fp:
                        M_out[ix,iy]=1
    print("Sanity Check ", np.sum(M_out==3),np.sum(M_ground>M_out), np.sum(M_out>M_ground)-np.sum(M_out==3))    
    WriteTfile(outputfile,M_out,groundfile)
                    
def expandM(M_input,x_dim,y_dim): #xdim,ydim hasto be larger than matrix's dimensions
    input_x_dim=M_input.shape[0]
    input_y_dim=M_input.shape[1]
    
    M_out=np.zeros((x_dim,y_dim))
    M_out[:input_x_dim,:input_y_dim]=M_input.copy()
    sel=np.random.choice(input_y_dim,y_dim-input_y_dim, replace=True)
    for i in range(y_dim-input_y_dim):
        M_out[:input_x_dim,input_y_dim+i]=M_input[:,sel[i]].copy()
    sel=np.random.choice(input_x_dim,x_dim-input_x_dim, replace=True)
    for i in range(x_dim-input_x_dim):
        M_out[input_x_dim+i,:]=M_out[sel[i],:].copy()
    
    return M_out

        
def AddDoublet(DoubPer,groundfile,inputfilename,outputfilename): # adds doublets to a supplied input file with a similar rate of fn.fp,na and writes it to output file. 
    
    outputfilename=inputfilename
    M_in=ReadFasis(inputfilename)
    M_g=ReadFasis(groundfile)
    f_n=np.sum((M_in==0)*(M_g==1))/np.sum(M_g==1)
    f_p=np.sum((M_in==1)*(M_g==0))/np.sum(M_g==0)
    f_na=np.sum(M_in==3)/(M_g.shape[0]*M_g.shape[1])
    M_out=M_in.copy()
    Dperc=DoubPer
    cellcount=M_in.shape[0]
    remainingcells=[i for i in range(cellcount)]
    n_doublets=int(Dperc*cellcount)
    print(inputfilename, " Observed FN/FP/NA rates : ", f_n,f_p,f_na)
    added_fp=0
    added_fn=0
    for i in range(n_doublets):
        i1=np.random.choice(remainingcells)
        remainingcells.remove(i1)
        i2=np.random.choice(remainingcells)
        remainingcells.remove(i2)
#            print(M_out[i1,:])
        M_out[i1,:]=(M_g[i1,:]+M_g[i2,:])>0
#    print(M_g[i1,:]<M_out[i1,:])
#    print(M_out[i1,:],M_g[i1,:],M_g[i2,:])
        
        for j in range(M_g.shape[1]):
            if M_out[i1,j]==0:
                if np.random.rand()<f_p:
                    M_out[i1,j]=1
                    added_fp=added_fp+1
            if M_out[i1,j]==1:
                if np.random.rand()<f_n:
                    M_out[i1,j]=0
                    added_fn=added_fn+1
            if M_in[i1,j]==3:
                M_out[i1,j]=3
    f_n=np.sum((M_out==0)*(M_g==1))/np.sum(M_g==1)
    #f_p=np.sum((M_out==1)*(M_g==0))/np.sum(M_g==0)
    print(f, "Doublets added: ", n_doublets, f_n,added_fp,np.sum(M_in==3)==np.sum(M_out==3))
    WriteTfile(outputfilename,M_out,inputfilename)    
                
    
        
        
    


In [3]:
f_ground=os.listdir()

for f in f_ground:
    inputfile=f
    if f.endswith(".before_FP_FN_NA"):
        print(f)
        Pfn=float(inputfile.split("fn_")[1].split("-")[0])
        #Pfp=float(inputfile.split("fp_")[1].split("-")[0])
        Pfp=0.03
        #na=float(inputfile.split("-na_")[1].split("-")[0])
        na=0.05
        doub=0
        outname=f.split("fp_")[0]+ "fp_"+str(Pfp)+"-fn"+f.split("-fn")[1]
        outname=outname.split("na_")[0]+ "na_"+str(na)+"-d"+outname.split("-d")[1]
        outname=outname.split("d_")[0]+ "d_"+str(doub)+"-l"+outname.split("-l")[1]
        print(outname)
        os.rename(inputfile,outname)

simNo_1-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0-fn_0.05-na_0-d_0-l_1000000.SC.before_FP_FN_NA
simNo_1-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0.03-fn_0.05-na_0.05-d_0-l_1000000.SC.before_FP_FN_NA
simNo_1-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0-fn_0.1-na_0-d_0-l_1000000.SC.before_FP_FN_NA
simNo_1-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0.03-fn_0.1-na_0.05-d_0-l_1000000.SC.before_FP_FN_NA
simNo_1-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0-fn_0.2-na_0-d_0-l_1000000.SC.before_FP_FN_NA
simNo_1-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0.03-fn_0.2-na_0.05-d_0-l_1000000.SC.before_FP_FN_NA
simNo_10-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0-fn_0.05-na_0-d_0-l_1000000.SC.before_FP_FN_NA
simNo_10-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0.03-fn_0.05-na_0.05-d_0-l_1000000.SC.before_FP_FN_NA
simNo_10-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0-fn_0.1-na_0-d_0-l_1000000.SC.before_FP_FN_NA
simNo_10-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0.03-fn_0.1-na_0.05-d_0-l_1000000.S

In [ ]:
f_ground=os.listdir()

for f in f_ground:
    inputfile=f
    if f.endswith(".before_FP_FN_NA"):
        print(f)
        Pfn=float(inputfile.split("fn_")[1].split("-")[0])
        Pfp=float(inputfile.split("fp_")[1].split("-")[0])
        na=float(inputfile.split("-na_")[1].split("-")[0])
        doubper=float(inputfile.split("-d_")[1].split("-")[0])
        print("FN/FP/NA ",Pfn,Pfp,na)
        AddNAFNFP(na,Pfn,Pfp,inputfile,inputfile[:-16])
        print("Adding Doublets")
        AddDoublet(doubper,f,inputfile[:-16],inputfile[:-16])

simNo_1-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0.03-fn_0.05-na_0.05-d_0-l_1000000.SC.before_FP_FN_NA
FN/FP/NA  0.05 0.03 0.05
27749 62251
Required NA/FN/FP 4500 1387 1867  Number of ones in the matrix :  27749
Sanity Check  4500 1387 1867
Adding Doublets
simNo_1-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0.03-fn_0.05-na_0.05-d_0-l_1000000.SC  Observed FN/FP/NA rates :  0.049983783199394574 0.02999148608054489 0.05
simNo_1-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0.03-fn_0.05-na_0.05-d_0-l_1000000.SC.before_FP_FN_NA Doublets added:  0 0.049983783199394574 0 True
simNo_1-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0.03-fn_0.1-na_0.05-d_0-l_1000000.SC.before_FP_FN_NA
FN/FP/NA  0.1 0.03 0.05
27749 62251
Required NA/FN/FP 4500 2774 1867  Number of ones in the matrix :  27749
Sanity Check  4500 2774 1867
Adding Doublets
simNo_1-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0.03-fn_0.1-na_0.05-d_0-l_1000000.SC  Observed FN/FP/NA rates :  0.09996756639878915 0.02999148608054489 0.05
simNo_1-

In [6]:
f_ground=os.listdir()

for f in f_ground:
    inputfile=f
    if f.endswith(".before_FP_FN_NA"):
        
        Pfn=float(inputfile.split("fn_")[1].split("-")[0])
        #Pfp=float(inputfile.split("fp_")[1].split("-")[0])
        Pfp=0.005
        fppar="fp_"+str(Pfp)+"-"
        #outputfile=inputfile.split("fp")[0] + fppar + inputfile.split("fp")[1].split("-")[1]
        outputfile=inputfile[:-16]
        ndim=int(inputfile.split("-n_")[1].split("-")[0])
        mdim=int(inputfile.split("-m_")[1].split("-")[0])
        na=float(inputfile.split("-na_")[1].split("-")[0])
        Add_Pwise_NAFNFP(na,Pfn,Pfp,inputfile,outputfile)

simNo_2-s_10-m_100-h_1-minVAF_0.03-ISAV_0-n_300-fp_0-fn_0.1-na_0-d_0-l_1000000.SC.before_FP_FN_NA
6238 23762
Required NA/FN/FP  0 623 118
Sanity Check  0 639 116
simNo_2-s_10-m_100-h_1-minVAF_0.03-ISAV_0-n_300-fp_0-fn_0.2-na_0-d_0-l_1000000.SC.before_FP_FN_NA
6238 23762
Required NA/FN/FP  0 1247 118
Sanity Check  0 1211 126
simNo_2-s_10-m_200-h_1-minVAF_0.03-ISAV_0-n_300-fp_0-fn_0.2-na_0-d_0-l_1000000.SC.before_FP_FN_NA
14854 45146
Required NA/FN/FP  0 2970 225
Sanity Check  0 3024 217
simNo_2-s_10-m_300-h_1-minVAF_0.03-ISAV_0-n_300-fp_0-fn_0.05-na_0-d_0-l_1000000.SC.before_FP_FN_NA
15371 74629
Required NA/FN/FP  0 768 373
Sanity Check  0 808 390
simNo_4-s_100-m_300-h_1-minVAF_0.005-ISAV_0-n_300-fp_0.001-fn_0.2-na_0.05-d_0-l_1000000.SC.before_FP_FN_NA
2915 83485
Required NA/FN/FP  4320 583 417
Sanity Check  4312 545 410


In [48]:
f_ground=os.listdir()

for f in f_ground:
    inputfile=f
    if f.endswith(".before_FP_FN_NA"):
        k_na=float(inputfile.split("-na_")[1].split("-")[0])
        k_fp=float(inputfile.split("-fp_")[1].split("-")[0])
        k_fn=float(inputfile.split("-fn_")[1].split("-")[0])
        #k_fp=0.03
        print("NA: ", k_na, " FP: ", k_fp, "FN: ",k_fn)
        AddNAFNFP(k_na,k_fn,k_fp,f,f[:-16])
        

NA:  0.05  FP:  0.03 FN:  0.05
3329 84271
Required NA/FN/FP  4380 166 2528
Sanity Check  4380 166 2528
NA:  0.05  FP:  0.03 FN:  0.2
3324 83376
Required NA/FN/FP  4335 664 2501
Sanity Check  4335 664 2501


In [37]:
f="simNo_1-s_100-m_300-h_1-minVAF_0.005-ISAV_0-n_1000-fp_0.001-fn_0.05-na_0.05-d_0-l_1000000.SC.before_FP_FN_NA"
print(f.split("m_")[0] + "m_"+ str(400)+"-h_"+ f.split("m_")[1].split("h_")[1])
print(f.split("n_")[0] + "n_"+ str(5000)+"-fp"+ f.split("m_")[1].split("fp")[1])

simNo_1-s_100-m_400-h_1-minVAF_0.005-ISAV_0-n_1000-fp_0.001-fn_0.05-na_0.05-d_0-l_1000000.SC.before_FP_FN_NA
simNo_1-s_100-m_300-h_1-minVAF_0.005-ISAV_0-n_5000-fp_0.001-fn_0.05-na_0.05-d_0-l_1000000.SC.before_FP_FN_NA


In [47]:
M_nodes=Mr
p_fp=0.001
p_fn=0.2
distances=np.zeros(M_nodes.shape[0])
D10=((M_nodes==0).astype(int).dot(node_piv==1))
D11=((M_nodes==1).astype(int).dot(node_piv==1))
D00=((M_nodes==0).astype(int).dot(node_piv==0))
D01=((M_nodes==1).astype(int).dot(node_piv==0))

distances=-np.multiply(np.power(p_fp,D10),np.power(1-p_fp,D00))
distances=np.multiply(distances,np.power(p_fn,D01))
distances=np.multiply(distances,np.power(1-p_fn,D11))

NameError: name 'node_piv' is not defined